In [ ]:
%load_ext autoreload

%autoreload 2


image is too large

[Resize All Images in a Linux Folder](https://linuxhint.com/resize-all-images-folder-linux/)

In [ ]:
mkdir -p "../images"

for img in *.jpg 
do
    convert -resize 13.333% "$img" "../images/$img"
done

this can Ignore Aspect Ratio

In [ ]:
mkdir -p "../images"

for img in *.jpg 
do
    convert -resize 640x384\! "$img" "../images/$img"
done

## tfrecord create
this dataset is different from COCO, so we need other method to create tfrecord.  
To follow the same workflow, make dataset to COCO format seem to be a good method. And make it able to use by other framework.    
I find this tutorial [Convert any dataset to COCO object detection format with SAHI](https://medium.com/codable/convert-any-dataset-to-coco-object-detection-format-with-sahi-95349e1fe2b7).  

In [ ]:
python yolo2coco.py --root_dir "../obj/" --random_split --split_ratio "0.8:0.1:0.1"

and then as usual but small modification

In [ ]:

TRAIN_DATA_DIR='../obj/images'
TRAIN_ANNOTATION_FILE_DIR='../obj/annotations/train.json'
OUTPUT_TFRECORD_TRAIN='../cabbage_coco_tfrecords/train'

# Need to provide
  # 1. image_dir: where images are present
  # 2. object_annotations_file: where annotations are listed in json format
  # 3. output_file_prefix: where to write output convered TFRecords files
python -m official.vision.data.create_coco_tf_record --logtostderr \
  --image_dir=${TRAIN_DATA_DIR} \
  --object_annotations_file=${TRAIN_ANNOTATION_FILE_DIR} \
  --output_file_prefix=$OUTPUT_TFRECORD_TRAIN \
  --num_shards=1


In [ ]:

VALID_DATA_DIR='../obj/images'
VALID_ANNOTATION_FILE_DIR='../obj/annotations/val.json'
OUTPUT_TFRECORD_VALID='../cabbage_coco_tfrecords/valid'

python -m official.vision.data.create_coco_tf_record --logtostderr \
  --image_dir=$VALID_DATA_DIR \
  --object_annotations_file=$VALID_ANNOTATION_FILE_DIR \
  --output_file_prefix=$OUTPUT_TFRECORD_VALID \
  --num_shards=1


In [ ]:

TEST_DATA_DIR='../obj/images'
TEST_ANNOTATION_FILE_DIR='../obj/annotations/test.json'
OUTPUT_TFRECORD_TEST='../cabbage_coco_tfrecords/test'

python -m official.vision.data.create_coco_tf_record --logtostderr \
  --image_dir=$TEST_DATA_DIR \
  --object_annotations_file=$TEST_ANNOTATION_FILE_DIR \
  --output_file_prefix=$OUTPUT_TFRECORD_TEST \
  --num_shards=1


# start

In [ ]:
import os
import io
import pprint
import tempfile
import matplotlib
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from PIL import Image
from six import BytesIO
from IPython import display
from urllib.request import urlopen

tf.print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
import orbit
import tensorflow_models as tfm

from official.core import exp_factory
from official.core import config_definitions as cfg
from official.vision.serving import export_saved_model_lib # only served lib !
from official.vision.ops.preprocess_ops import normalize_image
from official.vision.ops.preprocess_ops import resize_and_crop_image
from official.vision.utils.object_detection import visualization_utils
from official.vision.dataloaders.tf_example_decoder import TfExampleDecoder

from official.vision.ops.nms import sorted_non_max_suppression_padded
# import official.projects.centernet.common.registry_imports
from official.projects.centernet.configs.centernet import centernet_hourglass_coco

import official.projects.yolo.common.registry_imports



pp = pprint.PrettyPrinter(indent=4) # Set Pretty Print Indentation
print(tf.__version__) # Check the version of tensorflow used

%matplotlib inline

print("Vision modules: ", dir(tfm.vision))

## Configure the Retinanet Resnet FPN COCO model for custom dataset.

if your image is not square e.g. 512x512, 256x256 you need to set more training step.  
for square image: 2000+ step  
for rectangle image: 3000+ step

for other model e.g. 'retinanet_spinenet_coco', 'retinanet_maxvit_coco', 'detr_coco', 'fasterrcnn_resnetfpn_coco', 'maskrcnn_resnetfpn_coco', 'pix2seq_r50_coco', 'centernet_hourglass_coco', 'deep_mask_head_rcnn_spinenet_coco', 'panoptic_deeplab_resnet_coco', 'panoptic_fpn_coco'  
'coco_yolov7'

plz refer [configs](https://github.com/tensorflow/models/tree/20cd431c7940489ad807706e046add604e43e75b/official/vision/configs) xxxnext_test.py or [repo:tensorflow/models register_config_factory](https://github.com/search?q=repo%3Atensorflow%2Fmodels%20register_config_factory&type=code) find regist name.


'centernet_hourglass_coco' is at official/projects/centernet/configs/centernet.py.

---

Or only model name in `official/vision/serving/detection_test.py` have pretrained model?


In [ ]:
# train_data_input_path = 'bccd_coco_tfrecords/train-00000-of-00001.tfrecord'
# valid_data_input_path = 'bccd_coco_tfrecords/valid-00000-of-00001.tfrecord'
# test_data_input_path = 'bccd_coco_tfrecords/test-00000-of-00001.tfrecord'
# model_dir = 'trained_model/'
# export_dir ='exported_model/'

# model_name='retinanet_spinenet_coco'
#model_name='maskrcnn_resnetfpn_coco' # no mask data
model_name='fasterrcnn_resnetfpn_coco'


train_data_input_path = '../cabbage_coco_tfrecords/train-00000-of-00001.tfrecord'
valid_data_input_path = '../cabbage_coco_tfrecords/valid-00000-of-00001.tfrecord'
test_data_input_path = '../cabbage_coco_tfrecords/test-00000-of-00001.tfrecord'

model_dir = '../cbg_{}_trained_model/'.format(model_name) #this path need to modify for each model #https://github.com/tensorflow/models/issues/9703
export_dir ='../cbg_{}_exported_model/'.format(model_name)



exp_config = exp_factory.get_exp_config(model_name)
# exp_config=centernet_hourglass_coco()

batch_size = 2 #8
num_classes = 1 #3

# HEIGHT, WIDTH = 256, 256
WIDTH, HEIGHT=640, 384 #288,384


In [ ]:

IMG_SIZE = [HEIGHT, WIDTH, 3]

# Backbone config.
exp_config.task.freeze_backbone = False
exp_config.task.annotation_file = ''

# Model config.
exp_config.task.model.input_size = IMG_SIZE
exp_config.task.model.num_classes = num_classes + 1
#this is for retinanet, maskrcnn have no this setting.
# exp_config.task.model.detection_generator.tflite_post_processing.max_classes_per_detection = exp_config.task.model.num_classes

# Training data config.
exp_config.task.train_data.input_path = train_data_input_path
exp_config.task.train_data.dtype = 'float32'
exp_config.task.train_data.global_batch_size = batch_size
exp_config.task.train_data.parser.aug_rand_hflip=True
exp_config.task.train_data.parser.aug_rand_vflip=True
exp_config.task.train_data.parser.aug_scale_max = 1.2
exp_config.task.train_data.parser.aug_scale_min = 0.8

# Validation data config.
exp_config.task.validation_data.input_path = valid_data_input_path
exp_config.task.validation_data.dtype = 'float32'
exp_config.task.validation_data.global_batch_size = batch_size

#--------------------------------------------
logical_device_names = [logical_device.name for logical_device in tf.config.list_logical_devices()]

if 'GPU' in ''.join(logical_device_names):
  print('This may be broken in Colab.')
  device = 'GPU'
elif 'TPU' in ''.join(logical_device_names):
  print('This may be broken in Colab.')
  device = 'TPU'
else:
  print('Running on CPU is slow, so only train for a few steps.')
  device = 'CPU'


train_steps = 5000
exp_config.trainer.steps_per_loop = 100 # steps_per_loop = num_of_training_examples // train_batch_size

exp_config.trainer.summary_interval = 100
exp_config.trainer.checkpoint_interval = 100
exp_config.trainer.validation_interval = 100
exp_config.trainer.validation_steps =  100 # validation_steps = num_of_validation_examples // eval_batch_size
exp_config.trainer.train_steps = train_steps
exp_config.trainer.optimizer_config.warmup.linear.warmup_steps = 100
exp_config.trainer.optimizer_config.learning_rate.type = 'cosine'
exp_config.trainer.optimizer_config.learning_rate.cosine.decay_steps = train_steps
exp_config.trainer.optimizer_config.learning_rate.cosine.initial_learning_rate = 0.1
exp_config.trainer.optimizer_config.warmup.linear.warmup_learning_rate = 0.05

#print rst
pp.pprint(exp_config.as_dict())
# display.Javascript('google.colab.output.setIframeHeight("500px");')

# distribution strategy
if exp_config.runtime.mixed_precision_dtype == tf.float16:
    tf.keras.mixed_precision.set_global_policy('mixed_float16')

if 'GPU' in ''.join(logical_device_names):
  distribution_strategy = tf.distribute.MirroredStrategy()
elif 'TPU' in ''.join(logical_device_names):
  tf.tpu.experimental.initialize_tpu_system()
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='/device:TPU_SYSTEM:0')
  distribution_strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
  print('Warning: this will be really slow.')
  distribution_strategy = tf.distribute.OneDeviceStrategy(logical_device_names[0])

print('Done')

# Create the Task object 
with distribution_strategy.scope():
  task = tfm.core.task_factory.get_task(exp_config.task, logging_dir=model_dir)



## Visualize a batch of the data.

In [ ]:
for images, labels in task.build_inputs(exp_config.task.train_data).take(1):
  print()
  print(f'images.shape: {str(images.shape):16}  images.dtype: {images.dtype!r}')
  print(f'labels.keys: {labels.keys()}')


In [ ]:
# category_index={
    # 1: {
    #     'id': 1,
    #     'name': 'Platelets'
    #    },
    # 2: {
    #     'id': 2,
    #     'name': 'RBC'
    #    },
    # 3: {
    #     'id': 3,
    #     'name': 'WBC'
    #    }
# }
category_index={
  1:{
    'id':1,
    'name':'Cabbage'
  }
}
tf_ex_decoder = TfExampleDecoder()


def show_batch(raw_records, num_of_examples):
  plt.figure(figsize=(20, 20))
  use_normalized_coordinates=True
  min_score_thresh = 0.30
  for i, serialized_example in enumerate(raw_records):
    plt.subplot(1, 3, i + 1)
    decoded_tensors = tf_ex_decoder.decode(serialized_example)
    image = decoded_tensors['image'].numpy().astype('uint8')
    scores = np.ones(shape=(len(decoded_tensors['groundtruth_boxes'])))
    visualization_utils.visualize_boxes_and_labels_on_image_array(
        image,
        decoded_tensors['groundtruth_boxes'].numpy(),
        decoded_tensors['groundtruth_classes'].numpy().astype('int'),
        scores,
        category_index=category_index,
        use_normalized_coordinates=use_normalized_coordinates,
        max_boxes_to_draw=200,
        min_score_thresh=min_score_thresh,
        agnostic_mode=False,
        instance_masks=None,
        line_thickness=1)

    plt.imshow(image)
    plt.axis('off')
    plt.title(f'Image-{i+1}')
  plt.show()



In [ ]:
buffer_size = 20
num_of_examples = 3

raw_records = tf.data.TFRecordDataset(
    exp_config.task.train_data.input_path).shuffle(
        buffer_size=buffer_size).take(num_of_examples)
show_batch(raw_records, num_of_examples)


# Train and evaluate.

In [ ]:
model, eval_logs = tfm.core.train_lib.run_experiment(
    distribution_strategy=distribution_strategy,
    task=task,
    mode='train_and_eval',
    params=exp_config,
    model_dir=model_dir,
    run_post_eval=True)


In [ ]:
%load_ext tensorboard
# %tensorboard --logdir '../cbg_hourglass_trained_model/' --port 6009
%tensorboard --logdir $model_dir --port 6013


In [ ]:
export_saved_model_lib.export_inference_graph(
    input_type='image_tensor',
    batch_size=1,
    input_image_size=[HEIGHT, WIDTH],
    params=exp_config,
    checkpoint_path=tf.train.latest_checkpoint(model_dir),
    export_dir=export_dir)


# inference


In [ ]:
def load_image_into_numpy_array(path):
  """Load an image from file into a numpy array.

  Puts image into numpy array to feed into tensorflow graph.
  Note that by convention we put it into a numpy array with shape
  (height, width, channels), where channels=3 for RGB.

  Args:
    path: the file path to the image

  Returns:
    uint8 numpy array with shape (img_height, img_width, 3)
  """
  image = None
  if(path.startswith('http')):
    response = urlopen(path)
    image_data = response.read()
    image_data = BytesIO(image_data)
    image = Image.open(image_data)
  else:
    image_data = tf.io.gfile.GFile(path, 'rb').read()
    image = Image.open(BytesIO(image_data))

  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (1, im_height, im_width, 3)).astype(np.uint8)



def build_inputs_for_object_detection(image, input_image_size):
  """Builds Object Detection model inputs for serving."""
  image, _ = resize_and_crop_image(
      image,
      input_image_size,
      padded_size=input_image_size,
      aug_scale_min=1.0,
      aug_scale_max=1.0)
  return image

In [ ]:
num_of_examples = 3

test_ds = tf.data.TFRecordDataset(
    test_data_input_path).take(
        num_of_examples)
show_batch(test_ds, num_of_examples)


## predict

In [ ]:


imported = tf.saved_model.load(export_dir)
model_fn = imported.signatures['serving_default']

input_image_size = (HEIGHT, WIDTH)
plt.figure(figsize=(20, 20))
min_score_thresh = 0.40 # Change minimum score for threshold to see all bounding boxes confidences.

for i, serialized_example in enumerate(test_ds):
  plt.subplot(1, 3, i+1)
  decoded_tensors = tf_ex_decoder.decode(serialized_example)
  image = build_inputs_for_object_detection(decoded_tensors['image'], input_image_size)
  image = tf.expand_dims(image, axis=0)
  image = tf.cast(image, dtype = tf.uint8)
  image_np = image[0].numpy()
  
  result = model_fn(image) #evaluate

  # nms_scores,nms_proposals=  sorted_non_max_suppression_padded(
  #   scores=result['detection_scores'], 
  #   boxes=result['detection_boxes'],
  #   max_output_size=100,
  #   iou_threshold=0.1 #cabbage impossible touch
  #   )
  #'boxes', 'classes', 'confidence'
  selected_indices=tf.image.non_max_suppression(
    boxes=result['detection_boxes'][0],
    scores=result['detection_scores'][0], 
    max_output_size=100,
    iou_threshold=0.01,
    score_threshold=min_score_thresh, #float('-inf'),
    name=None
  )
  nms_proposals = tf.gather(result['detection_boxes'][0], selected_indices)
  nms_scores=tf.gather(result['detection_scores'][0], selected_indices)
  nms_classes=tf.gather(result['detection_classes'][0], selected_indices)


  visualization_utils.visualize_boxes_and_labels_on_image_array(
      image_np,
      nms_proposals.numpy(),
      # result['detection_boxes'][0].numpy(),
      # selected_boxes

      # result['detection_classes'][0].numpy().astype(int),
      nms_classes.numpy().astype(int),

      # result['detection_scores'][0].numpy(),
      nms_scores.numpy(),

      category_index=category_index,
      use_normalized_coordinates=False,
      max_boxes_to_draw=100,
      min_score_thresh=min_score_thresh,
      agnostic_mode=False,
      instance_masks=None,
      line_thickness=2)
  plt.imshow(image_np)
  plt.axis('off')

plt.show()


# for test only

In [ ]:
# nms_scores
# nms_proposals
# result['detection_scores'][0]
# result.keys()
# # result['boxes']
# result['confidence']+0.6